# SA LLaMA

In [1]:
# installations for colab
#!pip install huggingface_hub transformers torch accelerate datasets peft urlextract

In [2]:
#!pip install -U bitsandbytes

In [ ]:
# huggingface login necessary because llama is gated
# token should be invalidated and refreshed after every git commit
from huggingface_hub import login
login(token="")

In [ ]:
import pandas as pd

df_tweets_bigtech_10k_sample = pd.read_csv("./tweets_bigtech_10ksample.csv")
df_tweets_bigtech_10k_sample.head()

FileNotFoundError: [Errno 2] No such file or directory: './tweets_bigtech_10ksample.csv'

In [ ]:
# loading the model using peft to be able to fine-tune
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

model = AutoModelForSequenceClassification.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    llm_int8_enable_fp32_cpu_offload=True
)

lora_config = LoraConfig(
     task_type=TaskType.SEQ_CLS,
     inference_mode=False,
     r=8,
     lora_alpha=16,
     lora_dropout=0.1
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

KeyboardInterrupt: 

# Predicting with pure llama-chat

In [ ]:
pipeline = 
pipeline("The new Iphone is not good.")

[{'label': 'LABEL_0', 'score': 0.8708171844482422}]

In [ ]:
df_tweets_bigtech_10k_sample["predicted_sentiment"] = df_tweets_bigtech_10k_sample["text"].apply(lambda x: pipeline(x)[0]["label"])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
df_tweets_bigtech_10k_sample["predicted_sentiment"].value_counts()

,count
predicted_sentiment,
LABEL_1,8313
LABEL_0,1187


In [ ]:
# adapting predicted labels to fit format
df_tweets_bigtech_10k_sample["predicted_sentiment"] = df_tweets_bigtech_10k_sample["predicted_sentiment"].apply(lambda x: 2 if x == "LABEL_1" else 0)

<ipython-input-38-53d60b99633f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_bigtech_10k_sample["predicted_sentiment"] = df_tweets_bigtech_10k_sample["predicted_sentiment"].apply(lambda x: 1 if x == "LABEL_1" else 0)


In [ ]:
# removing neutral tweets, since model only predicted negative and positive
df_tweets_bigtech_10k_sample = df_tweets_bigtech_10k_sample[df_tweets_bigtech_10k_sample["sentiment"] != 1]

In [ ]:
df_tweets_bigtech_10k_sample.head()

,Unnamed: 0,created_at,file_name,followers,friends,group_name,location,retweet_count,screenname,search_query,text,twitter_id,username,polarity,partition_0,partition_1,sentiment,predicted_sentiment
0,0,2020-07-12 09:24:26,AMD,25,114,AMD,United Kingdom,0.0,moffphcgaming,#AMD,Been on holiday so back now. Gonna try get som...,1.282244e+18,🕹MoffPHC Gaming🕹,-0.3102,Technology,AMD,0,2
1,1,2020-07-12 08:16:45,AMD,1719,1,AMD,digitalocean,1.0,LinuxDreams,#AMD,RT @LinuxReviews: #Linux architect Linus Torva...,1.282227e+18,LinuxDreams,-0.3612,Technology,AMD,0,0
2,2,2020-07-12 08:11:41,AMD,69,135,AMD,Amsterdam,1.0,LinuxReviews,#AMD,"#Linux architect Linus Torvalds: AVX512 Is ""A ...",1.282226e+18,LinuxReviews,-0.3612,Technology,AMD,0,2
3,3,2020-07-12 02:22:50,AMD,34,155,AMD,San Francisco,0.0,NdrewGarcia,#AMD,#AMD stuck in a range box chart https://t.co/5...,1.282138e+18,Encino_Man,-0.2500,Technology,AMD,0,2
4,4,2020-07-11 23:58:44,AMD,802,730,AMD,"New Jersey, USA",0.0,Roger_Clinton1,#AMD,$AMD Epyc Milan Leak – Three early Genesis sam...,1.282102e+18,Roger Ocasio-Clinton,-0.3400,Technology,AMD,0,2


In [ ]:
df_tweets_bigtech_10k_sample["sentiment"].value_counts()

,count
sentiment,
0,3500
2,3500


In [ ]:
df_tweets_bigtech_10k_sample["predicted_sentiment"].value_counts()

,count
predicted_sentiment,
2,6021
0,979


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def performance_metrics(df, label, prediction):
     accuracy = accuracy_score(df[label], df[prediction])
     precision = precision_score(df[label], df[prediction], average="weighted")
     recall = recall_score(df[label], df[prediction], average="weighted")
     f1 = f1_score(df[label], df[prediction], average="weighted")

     print(f"Accuracy: {accuracy}")
     print(f"Precision: {precision}")
     print(f"Recall: {recall}")
     print(f"F1-Score: {f1}")

performance_metrics(df_tweets_bigtech_10k_sample, "sentiment", "predicted_sentiment")

Accuracy: 0.5095714285714286
Precision: 0.5198912251111576
Recall: 0.5095714285714286
F1-Score: 0.43648145124446003


# Fine-tuning LLaMA

In [ ]:
#converting polarity to sentiment and trying to get llama to predict three labels (negative, neutral, positive)
def polarity_to_sentiment(polarity):
    if -0.2 <= polarity <= 0.2:
        return 1
    if polarity > 0.2:
        return 2
    else:
        return 0

df_tweets_bigtech_10k_sample["sentiment"] = df_tweets_bigtech_10k_sample["polarity"].apply(polarity_to_sentiment)
df_tweets_bigtech_10k_sample.head()

,Unnamed: 0,created_at,file_name,followers,friends,group_name,location,retweet_count,screenname,search_query,text,twitter_id,username,polarity,partition_0,partition_1,sentiment
0,0,2020-07-12 09:24:26,AMD,25,114,AMD,United Kingdom,0.0,moffphcgaming,#AMD,Been on holiday so back now. Gonna try get som...,1.282244e+18,🕹MoffPHC Gaming🕹,-0.3102,Technology,AMD,0
1,1,2020-07-12 08:16:45,AMD,1719,1,AMD,digitalocean,1.0,LinuxDreams,#AMD,RT @LinuxReviews: #Linux architect Linus Torva...,1.282227e+18,LinuxDreams,-0.3612,Technology,AMD,0
2,2,2020-07-12 08:11:41,AMD,69,135,AMD,Amsterdam,1.0,LinuxReviews,#AMD,"#Linux architect Linus Torvalds: AVX512 Is ""A ...",1.282226e+18,LinuxReviews,-0.3612,Technology,AMD,0
3,3,2020-07-12 02:22:50,AMD,34,155,AMD,San Francisco,0.0,NdrewGarcia,#AMD,#AMD stuck in a range box chart https://t.co/5...,1.282138e+18,Encino_Man,-0.2500,Technology,AMD,0
4,4,2020-07-11 23:58:44,AMD,802,730,AMD,"New Jersey, USA",0.0,Roger_Clinton1,#AMD,$AMD Epyc Milan Leak – Three early Genesis sam...,1.282102e+18,Roger Ocasio-Clinton,-0.3400,Technology,AMD,0


In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df_tweets_bigtech_10k_sample)

## fine-tuning with Tweets BigTech

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=3 # specifying num_labels resolves runtime CUDA error
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# pad token definitions for both tokenizer and model allow for batch size > 1
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("/content/tweets_bigtech_20ksample.csv")
df.dropna(inplace=True)

# model expects column explicitly called "labels"
df = df.rename({"sentiment" : "labels"}, axis=1)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/tweets_bigtech_20ksample.csv'

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,  # rank of the low-rank adaptation
    lora_alpha=32,  # scaling factor
    target_modules=["q_proj"],#, "v_proj"],  # target modules for LoRA
    lora_dropout=0.1,  # dropout rate
    bias="none",  # bias handling
    task_type="SEQ_CLS" # sentiment analysis is a form of sequence classification
)

model = get_peft_model(model, lora_config)

In [ ]:
from transformers import TrainingArguments

# experimenting with different training argument definitions

# batch size >= 64 results in out of memory error even with best GPU (A100)
training_args = TrainingArguments(
    output_dir="./llama2-sentiment-analysis",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=5e-5,
    fp16=True,
    save_total_limit=2
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.
        """
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs) # removing num_items_in_batch
        # saving past state if it exists
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = self.label_smoother(outputs, labels)
        else:
            # not using .loss here since the model may return tuples instead of ModelOutput
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        return (loss, outputs) if return_outputs else loss

In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

NameError: name 'training_args' is not defined

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: hausbichler-georg (hausbichler-georg-wirtschaftsuniversit-t-wien) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.926300,0.907115
2,0.783400,0.783130
3,0.635000,0.715672


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=375, training_loss=0.9057147407531738, metrics={'train_runtime': 8936.4734, 'train_samples_per_second': 5.371, 'train_steps_per_second': 0.042, 'total_flos': 9.55587206578176e+17, 'train_loss': 0.9057147407531738, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./llama2-sentiment-analysis-finetuned")
tokenizer.save_pretrained("./llama2-sentiment-analysis-finetuned")

('./llama2-sentiment-analysis-finetuned/tokenizer_config.json',
 './llama2-sentiment-analysis-finetuned/special_tokens_map.json',
 './llama2-sentiment-analysis-finetuned/tokenizer.model',
 './llama2-sentiment-analysis-finetuned/added_tokens.json',
 './llama2-sentiment-analysis-finetuned/tokenizer.json')

### applying the model

In [ ]:
# loading the model and applying it
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

llama_ft = AutoModelForSequenceClassification.from_pretrained(
    "./llama2-sentiment-analysis-finetuned",
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=3 # specifying num_labels resolves runtime CUDA error
)

tokenizer_llama_ft = AutoTokenizer.from_pretrained("./llama2-sentiment-analysis-finetuned")

# moving model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
llama_ft.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_fe

In [ ]:
df_tweets_bigtech_test = pd.DataFrame(test_dataset)

In [ ]:
tokens = tokenizer_llama_ft(df_tweets_bigtech_test["text"].tolist(), padding=True, truncation=True, return_tensors="pt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokens = {key: value.to(device) for key, value in tokens.items()}

In [ ]:
llama_ft.config.pad_token_id = llama_ft.config.eos_token_id

In [ ]:
import torch

# setting the model to evaluation-mode
llama_ft.eval()

batch_size = 32
num_batches = len(df_tweets_bigtech_test) // batch_size + 1
predictions = []

# performing evaluation in batches to not overload GPU
for i in range(num_batches):
  start_idx = i * batch_size
  end_idx = min((i + 1) * batch_size, len(df_tweets_bigtech_test))
  batch_tokens = tokenizer_llama_ft(df_tweets_bigtech_test["text"][start_idx:end_idx].tolist(), padding=True, truncation=True, return_tensors="pt")
  batch_tokens = {key: value.to(device) for key, value in batch_tokens.items()}

  with torch.no_grad():
      outputs = llama_ft(**batch_tokens)
      batch_predictions = torch.argmax(outputs.logits, dim=-1)
      predictions.extend(batch_predictions.cpu().numpy())

df_tweets_bigtech_test["llama_ft_prediction"] = predictions

NameError: name 'llama_ft' is not defined

In [14]:
def llama_pred(model, tokenizer, df, text_col):
    # setting padding tokens
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

    # moving model to GPU and setting it to evaluation-mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(model.device)
    model.eval()

    batch_size = 32
    num_batches = len(df) // batch_size + 1
    predictions = []

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(df))
        batch_tokens = tokenizer(df[text_col][start_idx:end_idx].tolist(), padding="max_length", truncation=True, return_tensors="pt", max_length=128)
        batch_tokens = {key: value.to(device) for key, value in batch_tokens.items()}

        with torch.no_grad():
            outputs = model(**batch_tokens)
            batch_predictions = torch.argmax(outputs.logits, dim=-1)
            predictions.extend(batch_predictions.cpu().numpy())

    df["llama_ft_prediction"] = predictions
    return df

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def performance_metrics(df, label, prediction):
     accuracy = accuracy_score(df[label], df[prediction])
     precision = precision_score(df[label], df[prediction], average="weighted")
     recall = recall_score(df[label], df[prediction], average="weighted")
     f1 = f1_score(df[label], df[prediction], average="weighted")

     print(f"Accuracy: {accuracy}")
     print(f"Precision: {precision}")
     print(f"Recall: {recall}")
     print(f"F1-Score: {f1}")

In [ ]:
performance_metrics(df_tweets_bigtech_test, "labels", "llama_ft_prediction")

## fine-tuning with brand sa data

In [ ]:
import pandas as pd
df_brd_tr = pd.read_csv("Dataset - Train.csv")
df_brd_tst = pd.read_csv("Dataset - Test.csv")
df_brd = pd.concat([df_brd_tr, df_brd_tst])
df_brd = df_brd.drop("Tweet", axis=1)
df_brd.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [ ]:
df_brd = df_brd[df_brd["tweet_text"].notna()]
len(df_brd)

8588

In [ ]:
df_brd["labels"] = df_brd["is_there_an_emotion_directed_at_a_brand_or_product"].replace({"Negative emotion" : 0, "Positive emotion" : 2, "No emotion toward brand or product" : 1, "I can't tell" : 1})

<ipython-input-12-4cedcaaa5b1b>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_brd["labels"] = df_brd["is_there_an_emotion_directed_at_a_brand_or_product"].replace({"Negative emotion" : 0, "Positive emotion" : 2, "No emotion toward brand or product" : 1, "I can't tell" : 1})


In [ ]:
import re
from urlextract import URLExtract
extractor = URLExtract()

def format_tweet(tweet):
    if not isinstance(tweet, str):
        return tweet
    # mask web urls
    urls = extractor.find_urls(tweet)
    for url in urls:
        tweet = tweet.replace(url, "{{URL}}")
    # format twitter account
    tweet = re.sub(r"\b(\s*)(@[\S]+)\b", r'\1{\2@}', tweet)
    return tweet

In [ ]:
df_brd["text"] = df_brd["tweet_text"].apply(format_tweet)

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df_brd)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/8588 [00:00<?, ? examples/s]

In [ ]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./llama2-sentiment-analysis",
    per_device_train_batch_size=40,
    per_device_eval_batch_size=40,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=500,
    learning_rate=5e-5,
    save_total_limit=2
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

<ipython-input-19-8f0087e0cc2e>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hausbichler-georg (hausbichler-georg-wirtschaftsuniversit-t-wien) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,No log,0.994654
2,No log,0.947128


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=86, training_loss=1.0313229228174963, metrics={'train_runtime': 16888.8092, 'train_samples_per_second': 0.814, 'train_steps_per_second': 0.005, 'total_flos': 2.7344831860703232e+17, 'train_loss': 1.0313229228174963, 'epoch': 2.0})

In [ ]:
model.save_pretrained("./llama2-sentiment-analysis-finetuned-brdsa")
tokenizer.save_pretrained("./llama2-sentiment-analysis-finetuned-brdsa")

('./llama2-sentiment-analysis-finetuned-brdsa/tokenizer_config.json',
 './llama2-sentiment-analysis-finetuned-brdsa/special_tokens_map.json',
 './llama2-sentiment-analysis-finetuned-brdsa/tokenizer.model',
 './llama2-sentiment-analysis-finetuned-brdsa/added_tokens.json',
 './llama2-sentiment-analysis-finetuned-brdsa/tokenizer.json')

In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.9471280574798584,
 'eval_runtime': 638.3091,
 'eval_samples_per_second': 2.691,
 'eval_steps_per_second': 0.067,
 'epoch': 2.0}

### applying the model

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from transformers import BitsAndBytesConfig
from peft import PeftModel

# first loading base model, then reloading fine-tuned model on top of it with PEFT
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForSequenceClassification.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=3
)

model_ft = PeftModel.from_pretrained(
    base_model,
    "llama-2-7b-sentiment-analysis-finetuned-brdsa"
)

tokenizer_ft = AutoTokenizer.from_pretrained("llama-2-7b-sentiment-analysis-finetuned-brdsa")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df_tweets_bigtech_10k = pd.read_csv("tweets_bigtech_10ksample.csv")
df_tweets_bigtech_10k.dropna(inplace=True)
df_tweets_bigtech_2k = df_tweets_bigtech_10k.sample(n=2000)
df_tweets_bigtech_2k.head()

,Unnamed: 0,created_at,file_name,followers,friends,group_name,location,retweet_count,screenname,search_query,text,twitter_id,username,polarity,partition_0,partition_1,sentiment
4869,4869,2020-07-08 23:45:43,Google,327,444,Google,Bangladesh,0.0,tusharrex,#Google,#Apply For #Google #Adsense and #Get #Approval...,1.281012e+18,novo chowdhury,0.0000,Technology,Google,1
2885,2885,2020-07-27 03:00:38,Netflix,2557,1085,Netflix,Sydney,0.0,OSullivanPaula,#Netflix,What @Econsultancy have been reading this week...,1.287584e+18,Paula O'Sullivan,-0.7184,Technology,Netflix,0
1371,1371,2020-08-10 06:47:08,Netflix,1907,4,Netflix,"Mumbai, India",0.0,moviestalkhindi,#Netflix,#SPIDERMANINTOTHESPIDERVERSE Spinoff Writer #B...,1.292714e+18,Movies Talk Official,-0.3612,Technology,Netflix,0
3340,3340,2020-07-20 18:37:53,Google,85,0,Google,Earth,3.0,iamanusmith,#Google,RT @Summertime_Tech: Next3 step @NOYBeu 👉 sto...,1.285283e+18,anusmith,-0.2960,Technology,Google,0
1965,1965,2020-07-17 22:16:06,Amazon,1985,3216,Amazon,Lincolnshire UK,132.0,ali18pot,#Amazon OR #AWS,RT @VikiDumee90: The Photo Shoot by Viki Dumee...,1.284251e+18,Alison Parkins,-0.3400,Technology,Amazon,0


In [ ]:
llama_pred(model_ft, tokenizer_ft, df_tweets_bigtech_2k, "text")

In [ ]:
performance_metrics(df_tweets_bigtech_2k, "sentiment", "llama_ft_prediction")

Accuracy: 0.2625
Precision: 0.29940289358718103
Recall: 0.2625
F1-Score: 0.20805419569455283


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

# second evaluation function to confirm accuracy of results
def evaluate_sentiment(df, true_col, pred_col):
    eval_df = df.dropna(subset=[true_col, pred_col])

    accuracy = accuracy_score(eval_df[true_col], eval_df[pred_col])
    precision, recall, f1, _ = precision_recall_fscore_support(
        eval_df[true_col],
        eval_df[pred_col],
        average='weighted'
    )

    print(classification_report(eval_df[true_col], eval_df[pred_col]))

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "num_samples": len(eval_df)
    }

metrics = evaluate_sentiment(df_tweets_bigtech_2k, "sentiment", "llama_ft_prediction")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"F1 Score: {metrics['f1']:.4f}")

              precision    recall  f1-score   support

           0       0.35      0.03      0.06       734
           1       0.25      0.64      0.36       535
           2       0.29      0.22      0.25       731

    accuracy                           0.26      2000
   macro avg       0.29      0.30      0.22      2000
weighted avg       0.30      0.26      0.21      2000

Accuracy: 0.2625
F1 Score: 0.2081


In [ ]:
llama_pred(model_ft, tokenizer_ft, df_brd, "text")

In [ ]:
performance_metrics(df_brd, "labels", "llama_ft_prediction")

Accuracy: 0.5846530041918957
Precision: 0.5381664117219317
Recall: 0.5846530041918957
F1-Score: 0.5525406933042277


## fine-tuning with sentiment corpus

In [4]:
import pandas as pd
df_sc = pd.read_json("sentiment_corpus.json", orient="records")
df_sc.head()

,text,labels
0,{{MENTION}} {{MENTION}} {{MENTION}} 226 x #tes...,1
1,{{MENTION}} Our fleet's on fleek. {{URL}} LMFA...,1
2,So many kitchen blenders are missing dick. #T...,0
3,#tesla. Not too rare anymore. {{URL}},1
4,iPhone users at #SXSW - any of you have your G...,0


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "meta-llama/Llama-2-7b-hf"
llama_t = AutoTokenizer.from_pretrained(model_name)
llama = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=3 # specifying num_labels resolves runtime CUDA error
)

llama_t.pad_token = llama_t.eos_token
llama.config.pad_token_id = llama.config.eos_token_id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

llama = prepare_model_for_kbit_training(llama)

lora_config = LoraConfig(
    r=16,  # rank of the low-rank adaptation
    lora_alpha=32,  # scaling factor
    target_modules=["k_proj", "q_proj", "v_proj", "o_proj"],  # target modules for LoRA
    lora_dropout=0.05,  # dropout rate
    bias="none",  # bias handling
    task_type="SEQ_CLS" # sentiment analysis is a form of sequence classification
)

llama = get_peft_model(llama, lora_config)

In [7]:
from datasets import Dataset

ds = Dataset.from_pandas(df_sc)

def tokenize_function(examples):
    return llama_t(examples["text"], padding="max_length", truncation=True, max_length=128)

In [8]:
tokenized_dataset = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/19915 [00:00<?, ? examples/s]

In [9]:
train_test_split1 = tokenized_dataset.train_test_split(test_size=0.15, seed=42)
train_val_dataset = train_test_split1['train']
test_dataset = train_test_split1['test']

train_test_split2 = train_val_dataset.train_test_split(test_size=0.15, seed=42)
train_dataset = train_test_split2['train']
val_dataset = train_test_split2['test']

In [10]:
from transformers import TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, precision_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

label_names = ["negative", "neutral", "positive"]

# batch size >= 64 results in out of memory error even with best GPU (A100)
training_args = TrainingArguments(
    output_dir="./llama2-sentiment-analysis",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=1e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    save_total_limit=2,
    label_names=label_names
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = CustomTrainer(
    model=llama,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=llama_t
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

{'eval_runtime': 47.12,
 'eval_samples_per_second': 63.413,
 'eval_steps_per_second': 1.995,
 'epoch': 1.9866666666666668}

In [ ]:
llama.save_pretrained("./llama2-sentiment-corpus")
llama_t.save_pretrained("./llama2-sentiment-corpus")

('./llama2-sentiment-corpus/tokenizer_config.json',
 './llama2-sentiment-corpus/special_tokens_map.json',
 './llama2-sentiment-corpus/tokenizer.model',
 './llama2-sentiment-corpus/added_tokens.json',
 './llama2-sentiment-corpus/tokenizer.json')

In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
from tqdm import tqdm

label_names = ["negative", "neutral", "positive"]

training_args = TrainingArguments(
    output_dir="./sentiment-llama-2-7b",
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    num_train_epochs=7,
    save_total_limit=7,
    label_names=label_names
)

trainer = Trainer(
    model=llama,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=llama_t
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-e3d32a2218db>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hausbichler-georg (hausbichler-georg-wirtschaftsuniversit-t-wien) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.750300,No log
2,0.565600,No log
3,0.497900,No log
4,0.342700,No log
5,0.202400,No log
6,0.035400,No log


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=784, training_loss=0.3978551819502395, metrics={'train_runtime': 5353.1125, 'train_samples_per_second': 18.813, 'train_steps_per_second': 0.146, 'total_flos': 4.9811950289289216e+17, 'train_loss': 0.3978551819502395, 'epoch': 6.942222222222222})

In [12]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from transformers import BitsAndBytesConfig
from peft import PeftModel

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_llama = AutoModelForSequenceClassification.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=3
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
best_f1 = 0
best_epoch = 0

val_df = val_dataset.to_pandas()

id_to_sentiment = {0: "negative", 1: "neutral", 2: "positive"}

folders = [113, 226, 339, 452, 565, 678, 784]
for epoch in range(7):
    foldernr = folders[epoch]
    checkpoint_dir = f"./sentiment-llama-2-7b/checkpoint-{str(foldernr)}"
    print(f"Loading model from {checkpoint_dir}")

    model = PeftModel.from_pretrained(
    base_llama,
    checkpoint_dir)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)

    val_df_copy = val_df.copy()
    val_df_with_preds = llama_pred(
        model,
        tokenizer,
        val_df_copy,
        text_col="text"
    )

    true_labels = val_df_with_preds["labels"].tolist()
    pred_labels = val_df_with_preds["llama_ft_prediction"].tolist()

    report = classification_report(
        true_labels,
        pred_labels,
        target_names=list(id_to_sentiment.values()),
        output_dict=True
    )

    accuracy = report["accuracy"]
    precision = report["weighted avg"]["precision"]
    recall = report["weighted avg"]["recall"]
    f1 = report["weighted avg"]["f1-score"]

    print(f"Epoch {epoch+1}: Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")

    print("\nPer-class metrics:")
    for sentiment_class in id_to_sentiment.values():
        print(f"{sentiment_class}: F1={report[sentiment_class]['f1-score']:.4f}, " +
              f"Precision={report[sentiment_class]['precision']:.4f}, " +
              f"Recall={report[sentiment_class]['recall']:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_epoch = epoch

print(f"Best epoch according to F1-score: {best_epoch+1}, F1: {best_f1:.4f}")

Loading model from ./sentiment-llama-2-7b/checkpoint-113
Epoch 1: Accuracy=0.7067, Precision=0.7100, Recall=0.7067, F1=0.7011

Per-class metrics:
negative: F1=0.7622, Precision=0.6818, Recall=0.8640
neutral: F1=0.6226, Precision=0.7192, Recall=0.5488
positive: F1=0.7193, Precision=0.7272, Recall=0.7115
Loading model from ./sentiment-llama-2-7b/checkpoint-226
Epoch 2: Accuracy=0.7677, Precision=0.7755, Recall=0.7677, F1=0.7674

Per-class metrics:
negative: F1=0.8170, Precision=0.7789, Recall=0.8591
neutral: F1=0.7259, Precision=0.6922, Recall=0.7631
positive: F1=0.7610, Precision=0.8515, Recall=0.6878
Loading model from ./sentiment-llama-2-7b/checkpoint-339
Epoch 3: Accuracy=0.7965, Precision=0.7966, Recall=0.7965, F1=0.7952

Per-class metrics:
negative: F1=0.8313, Precision=0.8354, Recall=0.8272
neutral: F1=0.7372, Precision=0.7770, Recall=0.7012
positive: F1=0.8170, Precision=0.7793, Recall=0.8586
Loading model from ./sentiment-llama-2-7b/checkpoint-452
Epoch 4: Accuracy=0.8142, Preci

### applying the model

### after 2 epochs

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from transformers import BitsAndBytesConfig
from peft import PeftModel

# first loading base model, then reloading fine-tuned model on top of it with PEFT
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForSequenceClassification.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=3
)

model_corpus_ft = PeftModel.from_pretrained(
    base_model,
    "./llama2-sentiment-corpus"
)

tokenizer_corpus_ft = AutoTokenizer.from_pretrained("./llama2-sentiment-corpus")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
df_corpus_test = test_dataset.to_pandas()

In [ ]:
llama_pred(model_corpus_ft, tokenizer_corpus_ft, df_corpus_test, "text")

In [23]:
# extended function for performance metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

def performance_metrics(df, label, prediction):
    accuracy = accuracy_score(df[label], df[prediction])
    precision = precision_score(df[label], df[prediction], average="weighted")
    recall = recall_score(df[label], df[prediction], average="weighted")
    f1 = f1_score(df[label], df[prediction], average="weighted")

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

    print("\nClassification Report:")
    label_names = ["negative", "neutral", "positive"]
    report = classification_report(df[label], df[prediction], target_names=label_names, digits=4)
    print(report)

In [ ]:
performance_metrics(df_corpus_test, "labels", "llama_ft_prediction")

Accuracy: 0.7162
Precision: 0.7192
Recall: 0.7162
F1-Score: 0.7159

Classification Report:
              precision    recall  f1-score   support

    negative     0.7244    0.7885    0.7551       960
     neutral     0.6625    0.6944    0.6781       998
    positive     0.7692    0.6699    0.7161      1030

    accuracy                         0.7162      2988
   macro avg     0.7187    0.7176    0.7164      2988
weighted avg     0.7192    0.7162    0.7159      2988



In [ ]:
df_corpus_test.head()

,text,labels,input_ids,attention_mask,llama_ft_prediction
0,iPhone battery is going quickly. Guy behind me...,0,"[1, 18483, 16988, 338, 2675, 9098, 29889, 1588...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,Oops that's going to kill many people's dreams...,0,"[1, 438, 3554, 393, 29915, 29879, 2675, 304, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
2,RT {{MENTION}} #BREAKING:Are they SERIOUSLY st...,0,"[1, 390, 29911, 8620, 13780, 2725, 930, 396, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
3,Get Death Stranding with NVIDIA RTX NOW! {{UR...,0,"[1, 3617, 14450, 3767, 392, 292, 411, 405, 130...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
4,{{MENTION}} - #Apple is the classiest fascist ...,0,"[1, 8620, 13780, 2725, 930, 448, 396, 2052, 28...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2


### after new method with checkpointing

In [16]:
# first loading base model, then reloading fine-tuned model on top of it with PEFT
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_llama = AutoModelForSequenceClassification.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=3
)

llama_corpus_4 = PeftModel.from_pretrained(
    base_llama,
    "./sentiment-llama-2-7b/checkpoint-452"
)

llama_corpus_4_t = AutoTokenizer.from_pretrained("./sentiment-llama-2-7b/checkpoint-452")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
llama_pred(llama_corpus_4, llama_corpus_4_t, df_corpus_test, "text")

,text,labels,input_ids,attention_mask,llama_ft_prediction
0,iPhone battery is going quickly. Guy behind me...,0,"[1, 18483, 16988, 338, 2675, 9098, 29889, 1588...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,Oops that's going to kill many people's dreams...,0,"[1, 438, 3554, 393, 29915, 29879, 2675, 304, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
2,RT {{MENTION}} #BREAKING:Are they SERIOUSLY st...,0,"[1, 390, 29911, 8620, 13780, 2725, 930, 396, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
3,Get Death Stranding with NVIDIA RTX NOW! {{UR...,0,"[1, 3617, 14450, 3767, 392, 292, 411, 405, 130...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
4,{{MENTION}} - #Apple is the classiest fascist ...,0,"[1, 8620, 13780, 2725, 930, 448, 396, 2052, 28...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
...,...,...,...,...,...
2983,"{{MENTION}} yes, I've been waiting for four ho...",0,"[1, 8620, 13780, 2725, 930, 4874, 29892, 306, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
2984,"""SEOs love Google Search Console - it has the ...",2,"[1, 376, 1660, 24768, 5360, 5087, 11856, 9405,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
2985,{{MENTION}} ready to go to Disneyworld! For t...,2,"[1, 8620, 13780, 2725, 930, 7960, 304, 748, 30...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2986,😡 wtf was #netflix thinking indont like that a...,0,"[1, 29871, 243, 162, 155, 164, 281, 13264, 471...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0


In [24]:
performance_metrics(df_corpus_test, "labels", "llama_ft_prediction")

Accuracy: 0.7972
Precision: 0.7966
Recall: 0.7972
F1-Score: 0.7968

Classification Report:
              precision    recall  f1-score   support

    negative     0.8196    0.8375    0.8284       960
     neutral     0.7667    0.7375    0.7518       998
    positive     0.8042    0.8175    0.8108      1030

    accuracy                         0.7972      2988
   macro avg     0.7968    0.7975    0.7970      2988
weighted avg     0.7966    0.7972    0.7968      2988

